In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from tqdm import trange, tqdm, tqdm_notebook

In [2]:
bus_bucheon = pd.read_csv('output/bus_operate_num.csv')
bus_bucheon = bus_bucheon.iloc[:, range(1,6)]
display(bus_bucheon.head(5))

,정류소id,val,정류소명,위도,경도
0,210000001.0,164.0,삼산체육관역.영상문화단지정문,37.507767,126.742100
1,210000003.0,784.0,KT부천지사사거리,37.485933,126.776467
2,210000004.0,935.0,부천대학교,37.488683,126.777033
3,210000005.0,935.0,부천소방서.부천대학사거리,37.491500,126.777683
4,210000006.0,572.0,기업은행.위브더스테이트5단지,37.498350,126.777217


In [3]:
bike_bucheon = pd.read_csv('output/bike_bucheon.csv')
bike_bucheon = bike_bucheon.iloc[:, range(1,4)]
display(bike_bucheon)

,자전거대여소명,위도,경도
0,공공자전거대여소 부천시청역,37.504938,126.763465
1,공공자전거대여소 상동역,37.506069,126.754122
2,공공자전거대여소 신중동역,37.503068,126.775751
3,공공자전거대여소 부천역,37.484717,126.782677
4,공공자전거대여소 송내역,37.488286,126.755075
5,공공자전거대여소 부천시청,37.503728,126.766102


In [4]:
school = pd.read_csv('./data/clean/school.csv')
school = school.iloc[:, range(1,5)]
display(school.head())

,이름,도로명주소,위도,경도
0,상인초등학교,경기도 부천시 도약로 61,37.512737,126.756481
1,상동초등학교,경기도 부천시 부흥로 70,37.496878,126.750145
2,원미초등학교,경기도 부천시 부흥로 423,37.489321,126.787470
3,부천동초등학교,경기도 부천시 지봉로51번길 22,37.488088,126.804290
4,상지초등학교,경기도 부천시 장말로 189,37.493609,126.763526


In [5]:
temp = gpd.read_file('data/bucheongrid/bucheonjsonjson.geojson')
temp = temp.dropna(axis=0)
축척 = sum([
    *(temp['geometry'].bounds['maxx'] - temp['geometry'].bounds['minx']),
    *(temp['geometry'].bounds['maxy'] - temp['geometry'].bounds['miny']),
    ]) / (2 * len(temp)) / 100
도보속도 = 133.1

In [6]:
minute = 5
거리 = minute * 133.1 / 100 * 60

In [7]:
def multipolygon_to_경도(x):
    경도, 위도 = x[0].exterior.xy 
    return np.array([x for x in 경도])
def multipolygon_to_위도(x):
    경도, 위도 = x[0].exterior.xy 
    return np.array([x for x in 위도])

In [8]:
def get_within_nminutes_infor(Data_input, Data_infor_input, name, value=False, geojson=False):
    """
    array을 이용하여 한번에 거리를 계산 후, 도보 3분이내 거리안에 해당하는 경우의
    수, 거리의 최소,평균,최대, 값의 최소,평균,최대를 계산합니다.
    
    geojson파일인 경우 멀티폴리곤에서 각각 경위도를 계산해주고 작업을 시행합니다.
    """
    
    Data = Data_input.copy()
    Data_infor = Data_infor_input.copy()
    
    print(name + "_작업중.............")
    
    ## geojson의 경우.. 지적도의 경우 경위도를 만들어 넣어 줄 거기 때문에 try로 함수입력
    if geojson:
        try:
            Data_infor['경도'] = Data_infor['geometry'].apply(multipolygon_to_경도) 
            Data_infor['위도'] = Data_infor['geometry'].apply(multipolygon_to_위도)        
        except:
            try:
                Data_infor['경도'] = Data_infor['geometry'].centroid.x
                Data_infor['위도'] = Data_infor['geometry'].centroid.y
            except:
                pass
            
    Len1 = len(Data)
    Len2 = len(Data_infor)
    
    tt_위도 = np.repeat(Data['위도'], Len2)
    tt_위도 = np.array(tt_위도).reshape(Len1, Len2)
    diff_위도 = tt_위도 - np.array(Data_infor['위도'])

    tt_경도 = np.repeat(Data['경도'], Len2)
    tt_경도 = np.array(tt_경도).reshape(Len1, Len2)
    diff_경도 = tt_경도 - np.array(Data_infor['경도'])
    
    diff_위도 = diff_위도**2
    diff_경도 = diff_경도**2
    dist = (diff_위도 + diff_경도)**(0.5)
    
    
    Count = []
    Dist_min = []
    Dist_mean = []
    Dist_max = []
    
    Val_min = []
    Val_mean = []
    Val_max = []
    
    
    
    for idx in tqdm(Data.index):
        if geojson:
            dist_idx = np.array([i.min() for i in dist[idx]])
        else:
            dist_idx = dist[idx]
        
        temp = Data_infor.loc[dist_idx < 거리 * 축척]           
        Count.append(len(temp))
        
        try:
            Dist_min.append(dist_idx[dist_idx < 거리 * 축척].min())
            Dist_mean.append(dist_idx[dist_idx < 거리 * 축척].mean())
            Dist_max.append(dist_idx[dist_idx < 거리 * 축척].max())
        except:
            ## 수정된 부분!
            Dist_min.append(0.0025)
            Dist_mean.append(0.0025)
            Dist_max.append(0.0025)
            
    Data[name + "_" + str(minute) + "분이내수"] = Count
    Data[name + "_" + str(minute) + "분이내거리최소"] = Dist_min
    Data[name + "_" + str(minute) + "분이내거리평균"] = Dist_mean
    Data[name + "_" + str(minute) + "분이내거리최대"] = Dist_max
        
    
    if value:
        for idx in tqdm(Data.index):
            if geojson:
                dist_idx = np.array([i.min() for i in dist[idx]])
            else:
                dist_idx = dist[idx]
                
            temp = Data_infor.loc[dist_idx < 거리 * 축척]
            Val_min.append(temp[value].min())
            Val_mean.append(temp[value].mean())
            Val_max.append(temp[value].max())
            
        Data[name + "_" + str(minute) + "분이내값최소"] = Val_min
        Data[name + "_" + str(minute) + "분이내값평균"] = Val_mean
        Data[name + "_" + str(minute) + "분이내값최대"] = Val_max
        Data = Data.fillna(0)
        
        return Data
    
    else:
        return Data

In [9]:
school_result = get_within_nminutes_infor(Data_input = bike_bucheon, Data_infor_input = school, name="학교")

100%|██████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 1744.84it/s]

학교_작업중.............


In [10]:
bus_result = get_within_nminutes_infor(Data_input = bike_bucheon, Data_infor_input = bus_bucheon, name="버스", value = 'val')

100%|███████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 225.23it/s]

버스_작업중.............


In [11]:
school_result.head()

,자전거대여소명,위도,경도,학교_5분이내수,학교_5분이내거리최소,학교_5분이내거리평균,학교_5분이내거리최대
0,공공자전거대여소 부천시청역,37.504938,126.763465,3,0.002004,0.002762,0.003911
1,공공자전거대여소 상동역,37.506069,126.754122,0,0.002500,0.002500,0.002500
2,공공자전거대여소 신중동역,37.503068,126.775751,4,0.003406,0.003680,0.003897
3,공공자전거대여소 부천역,37.484717,126.782677,1,0.003312,0.003312,0.003312
4,공공자전거대여소 송내역,37.488286,126.755075,0,0.002500,0.002500,0.002500


In [12]:
bus_result.head()

,자전거대여소명,위도,경도,버스_5분이내수,버스_5분이내거리최소,버스_5분이내거리평균,버스_5분이내거리최대,버스_5분이내값최소,버스_5분이내값평균,버스_5분이내값최대
0,공공자전거대여소 부천시청역,37.504938,126.763465,12,0.001450,0.002667,0.004016,87.0,655.500000,1170.0
1,공공자전거대여소 상동역,37.506069,126.754122,21,0.000541,0.002512,0.003993,56.0,462.809524,1359.0
2,공공자전거대여소 신중동역,37.503068,126.775751,14,0.001638,0.003099,0.003914,77.0,430.142857,1196.0
3,공공자전거대여소 부천역,37.484717,126.782677,19,0.001424,0.002564,0.004012,91.0,713.315789,1697.0
4,공공자전거대여소 송내역,37.488286,126.755075,14,0.000307,0.002400,0.003917,84.0,676.428571,1903.0


In [13]:
school_result.to_csv('output/var_school.csv')

In [14]:
bus_result.to_csv('output/var_bus.csv')